In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
import numpy as np
from data_work import data_split
from data_work import processed
from data_work import analyse_dataset
from data_work import set_data
from sklearn.model_selection import train_test_split
import pandas as pd
data=pd.read_csv("C:\\NCKH\\FLMODEL\\datanonid.csv",index_col=None)


In [3]:
id=[16,17,21,22,23,24]
col=data.columns
for idx in id:
    data=data.drop(col[idx],axis=1)
data=data.drop(columns='attack')
data=data.drop(columns='category')
data = processed(data,"subcategory ")

In [4]:

traindata,test_data=train_test_split(data,test_size=0.2,random_state=42,stratify=data['subcategory '])
data = traindata.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
client_data,server_data=train_test_split(traindata,test_size=0.001,random_state=42)
client_data = client_data.reset_index(drop=True)
server_data = server_data.reset_index(drop=True)

In [5]:
test_data.shape

(1406, 27)

In [6]:

testdata_list=set_data([test_data])[0]

#chia data thành list gồm 2 phàn tử [0] là X, [1] là y

len X 1406, len y 1406


# iid dataset setup 

In [7]:
from model import Net

##Create model function

In [8]:
import torch
import torch.nn as nn

In [9]:
trainloaders,testloaders=data_split(client_data,"subcategory ",8,0.1,IID=1)

prepairing IID dataset
len of client data :5614, len of each subdata : 701
client number 1 : train(631), val(70)
client number 2 : train(631), val(70)
client number 3 : train(631), val(70)
client number 4 : train(631), val(70)
client number 5 : train(631), val(70)
client number 6 : train(631), val(70)
client number 7 : train(631), val(70)
client number 8 : train(631), val(70)


In [64]:
names=["subcategory "]
a=analyse_dataset(trainloaders[7],names)
print(a)

thống kê nhãn subcategory :
3 62
1 67
2 62
6 62
7 68
4 75
5 69
0 9
[3, 1, 2, 6, 7, 4, 5, 0]


In [10]:
def semantic_backdoor(data,feature):
    threshold = data[feature].max()
    data.loc[data[feature] >= (threshold*0.5),"subcategory "]= 0
    return data

##Client

In [11]:
class client():
    def __init__(self,cid,net):
        self.cid=cid 
        self.model=net
        self.trainset=trainloaders[cid]
        self.client_label=analyse_dataset(trainloaders[cid],["subcategory "])
        self.discrete_columns=list(trainloaders[cid].columns)

    def fit(self ):
        # Read values from config
        self.model.fit(self.trainset, self.discrete_columns)
        return "ok"
    
    def Gen_fake_data(self,n_samples):
        synthetic_data = self.model.sample(n_samples)
        return synthetic_data



In [12]:
config = {
    "server_round": 3,
    "local_epochs": 5
    
}

In [13]:


def client_fn(cid: str,config): 
    server_round = config["server_round"]
    local_epochs = config["local_epochs"]
    net = Net(0.00)
    print(cid)
    return client(cid, net)

In [16]:
def data_backdoor(data,feature): 
    threshold = data[feature].max()
    print(threshold)
    filtered_df = data[data[feature] >= threshold*0.75]
    print(filtered_df[feature])
    X_backdoor,Y_backdoor=filtered_df.iloc[:, 1:].values,filtered_df.iloc[:, 0].values
    X_backdoor=torch.tensor((X_backdoor))
    Y_backdoor=Y_backdoor.squeeze().tolist()
    Y_backdoor=torch.tensor(Y_backdoor)
    return X_backdoor,Y_backdoor
X_backdoor,Y_backdoor=data_backdoor(data,'dur')


2917
3       2211
6       2789
7       2477
9       2446
11      2789
        ... 
5608    2471
5611    2297
5613    2757
5618    2485
5619    2262
Name: dur, Length: 1819, dtype: int64


##Server

In [27]:
import random
class server():
    def __init__(self):
        self.server_model=Net(0.00)
        self.trainset=server_data
        self.testset=testdata_list
        self.clients=[]
        n_clients=8
        config = {
            "server_round": round,
            "local_epochs": 5
        }
        for i in range(n_clients):
            client=client_fn(i,config)
            self.clients.append(client)

    def average_parameters(self, parameter_list):
        # Tính trung bình của các tensor trong danh sách
        avg_params = []
        for params in zip(*parameter_list):
            avg_param = torch.mean(torch.stack(params), dim=0)
            avg_params.append(avg_param)
        
        return avg_params



    #def fit_round():
    def simulation(self,rounds):
        
        for round in range(rounds):
            parameter=[]
            for i in range(len(self.clients)):
                #kết hợp data sever và xáo trộn
                data1 = pd.concat([self.clients[i].trainset, self.trainset], axis=0)
                print(f"training client {i} with {len(data1)} samples")
                data1=data1.sample(frac=1).reset_index(drop=True)
                # cập nhật tham số cho client
                if (round>0):
                    self.clients[i].model.load_parameter(self.server_model.get_parameter())
                # áp dụng backdoor
                if (i<4 and round == 4):
                    data1=semantic_backdoor(data1,'dur')
                X_train = data1.iloc[:, 1:].values
                y_train = data1.iloc[:, 0].values
                x_train=torch.tensor((X_train))
                y_train=y_train.squeeze().tolist()
                y_train=torch.tensor(y_train)
                #D_parameters,G_parameters=client.get_parameter()
                results=self.clients[i].model.fit(x_train.float(),y_train,0.003,0.2,15)
                client_params=self.clients[i].model.get_parameter()
                parameter.append(client_params)
            #train server model
            server_params=self.average_parameters(parameter)
            self.server_model.load_parameter(server_params)
            X_server = test_data.iloc[:, 1:].values
            y_server = test_data.iloc[:, 0].values
            x_server=torch.tensor((X_server))
            y_server=y_server.squeeze().tolist()
            y_server=torch.tensor(y_server)

            self.server_model.Cross_validation(x_server,y_server,5,0.003)

            #evaluate round accuracy
            _,accuracy=self.server_model.evaluate(self.testset[0],self.testset[1])
            print(f"round{round} accuracy:{accuracy}")
            _,accuracy_backdoor=self.server_model.evaluate(X_backdoor.float(),Y_backdoor)
            print(f"round{round} accuracy_backdoor:{accuracy_backdoor}")



            


In [25]:
fed=server()

0
thống kê nhãn subcategory :
3 71
1 94
7 94
2 82
5 95
4 87
6 97
0 11
1
thống kê nhãn subcategory :
7 98
1 87
6 89
2 100
5 98
4 73
3 79
0 7
2
thống kê nhãn subcategory :
7 95
4 84
1 99
3 88
6 86
5 83
2 82
0 14
3
thống kê nhãn subcategory :
3 92
4 92
2 92
0 12
1 92
6 86
7 82
5 83
4
thống kê nhãn subcategory :
1 83
2 88
4 97
5 84
3 95
6 99
7 79
0 6
5
thống kê nhãn subcategory :
7 98
2 97
1 78
5 96
3 76
4 86
6 90
0 10
6
thống kê nhãn subcategory :
3 82
5 95
7 85
6 85
1 85
0 6
4 101
2 92
7
thống kê nhãn subcategory :
6 73
7 98
5 89
3 89
1 101
2 79
4 87
0 15


In [26]:
fed.simulation(6)

training client 0 with 637 samples
accuracy of epoch 0 is 0.0, loss : 101.93902587890625
accuracy of epoch 1 is 0.1171875, loss : 370.4546813964844
accuracy of epoch 2 is 0.1328125, loss : 324.4346008300781
accuracy of epoch 3 is 0.2421875, loss : 57.281349182128906
accuracy of epoch 4 is 0.625, loss : 22.14596939086914
accuracy of epoch 5 is 0.40625, loss : 12.95762825012207
accuracy of epoch 6 is 0.5859375, loss : 6.048089504241943
accuracy of epoch 7 is 0.75, loss : 2.749345541000366
accuracy of epoch 8 is 0.953125, loss : 0.9211432337760925
accuracy of epoch 9 is 0.953125, loss : 0.7002255320549011
accuracy of epoch 10 is 0.9375, loss : 0.5519288778305054
accuracy of epoch 11 is 0.953125, loss : 0.5494894981384277
accuracy of epoch 12 is 0.9609375, loss : 0.27082446217536926
accuracy of epoch 13 is 0.9453125, loss : 0.30670273303985596
accuracy of epoch 14 is 0.9609375, loss : 0.27037209272384644
training client 1 with 637 samples
accuracy of epoch 0 is 0.171875, loss : 54.62125778

#Train models with IID and non-IID data  